In [37]:
import pandas as pd
import numpy as np
import os
import pickle
import time
import random
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import copy

from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.model_selection import StratifiedKFold

from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline as SkPipeline
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from imblearn.pipeline import Pipeline as ImbPipeline # Replaces sklearn Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

# load the dataset and fix values

In [38]:
# Use the GPU
if torch.backends.mps.is_available():
    print("MPS device is available.")
    device = torch.device("mps")
elif torch.cuda.is_available():
    print("CUDA device is available.")
    device = torch.device("cuda")
else:
    print("No GPU acceleration available.")
    device = torch.device("cpu")

# Fix the seed to have deterministic behaviour
def fix_random(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True  # slower

SEED = 1337
fix_random(SEED)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

DATASET_PATH = "dataset_train/dataset.csv"
dataset = pd.read_csv(DATASET_PATH, delimiter=",")

print(f"Shape of the dataset: {dataset.shape}")
duplicates = dataset[dataset.duplicated()]
print(f"Number of duplicates in the dataset: {duplicates.shape[0]}")

MPS device is available.
Shape of the dataset: (148301, 145)
Number of duplicates in the dataset: 0


## split the dataset

In [39]:
X = dataset.drop(columns=["grade"])
y = dataset["grade"].map({"A": 6, "B": 5, "C": 4, "D": 3, "E": 2, "F": 1, "G": 0})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [40]:
class NumericExtractor(BaseEstimator, TransformerMixin):
    """Extracts integers from strings using regex"""

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for col in X.columns:
            X[col] = X[col].astype(str).str.extract(r"(\d+)").astype(float)
        return X

class CyclicalDateEncoder(BaseEstimator, TransformerMixin):
    """Converts mm-yyyy to year + sine/cosine month encoding."""

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for col in X.columns:
            # errors="coerce" turns unparseable data/NaNs into NaT
            date_series = pd.to_datetime(X[col], format="%b-%Y", errors="coerce")
            # If date is NaT, these become NaN, which we handle in the pipeline later
            angle = 2 * np.pi * date_series.dt.month / 12

            X[f"{col}_year"] = date_series.dt.year
            X[f"{col}_month_sin"] = np.sin(angle)
            X[f"{col}_month_cos"] = np.cos(angle)
            
            X.drop(columns=[col], inplace=True)
        return X
    
class BinaryModeEncoder(BaseEstimator, TransformerMixin):
    """"Encodes 0 if value is mode, 1 if not"""
    def __init__(self):
        self.modes_ = {}

    def fit(self, X, y=None):
        # Calculate mode for each column and store it
        for col in X.columns:
            self.modes_[col] = X[col].mode()[0]
        return self

    def transform(self, X):
        X_copy = X.copy()
        for col, mode in self.modes_.items():
            # Apply: 1 if NOT the mode (least frequent), 0 if mode
            X_copy[col] = (X_copy[col] != mode).astype(int)
        return X_copy
    
class HighMissingDropper(BaseEstimator, TransformerMixin):
    """Drops columns with high missing percentage. Fits only on training data."""
    
    def __init__(self, threshold=20):
        self.threshold = threshold
        self.cols_to_drop_ = []

    def fit(self, X, y=None):
        missing_percentages = X.isna().mean() * 100
        self.cols_to_drop_ = missing_percentages[missing_percentages > self.threshold].index.tolist()
        return self

    def transform(self, X):
        X = X.copy()
        return X.drop(columns=self.cols_to_drop_)

In [41]:
redundant_cols = ["loan_title", "borrower_address_state"]
binary_cols = ["loan_payment_plan_flag", "listing_initial_status", "application_type_label",
               "hardship_flag_indicator", "disbursement_method_type", "debt_settlement_flag_indicator"]
one_hot_encoding_cols = ["borrower_housing_ownership_status", "borrower_income_verification_status",
                       "loan_status_current_code", "loan_purpose_category"]
extract_fields = ["loan_contract_term_months", "borrower_profile_employment_length"]
date_fields = ["loan_issue_date", "credit_history_earliest_line", "last_payment_date", "last_credit_pull_date"]
embed_column = ["borrower_address_zip"]

In [42]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import OrdinalEncoder
from sklearn.utils.class_weight import compute_class_weight

numeric_pipe = SkPipeline([
    ('extract', NumericExtractor()),
    ('impute', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipe = SkPipeline([
    ('impute', SimpleImputer(strategy='constant', fill_value='missing')),
    ('ohe', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
])

date_pipe = SkPipeline([
    ('cyclical', CyclicalDateEncoder()),
    ('impute', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()) # scaling is needed for the year, not for sin/cos
])

binary_pipe = SkPipeline([
    ('binary_enc', BinaryModeEncoder()), 
    ('impute', SimpleImputer(strategy='most_frequent'))
])

# remainder columns are numerical
remainder_pipe = SkPipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('num_pipe', numeric_pipe, extract_fields),
    ('cat_pipe', categorical_pipe, one_hot_encoding_cols),
    ('date_pipe', date_pipe, date_fields),
    ('bin_pipe', binary_pipe, binary_cols),
    ('drop_redundant', 'drop', redundant_cols),
    ],
    remainder=remainder_pipe
)

numerical_pipeline = SkPipeline([
    ('dropper', HighMissingDropper(threshold=20)),
    ('prep', preprocessor),
])

zip_pipeline = SkPipeline([
    ('impute', SimpleImputer(strategy='constant', fill_value='missing')),
    ('ordinal', OrdinalEncoder(dtype=np.int64, handle_unknown='use_encoded_value', unknown_value=-1))
])


# X_numerical_train = numerical_pipeline.fit_transform(X_train[numerical_columns], y_train)
# X_zip_train = zip_pipeline.fit_transform(X_train[embed_column]).squeeze()

# X_numerical_test = numerical_pipeline.transform(X_test[numerical_columns])
# X_zip_test = zip_pipeline.transform(X_test[embed_column]).squeeze()

# print(X_numerical_train.shape)
# print(X_zip_train.shape)
# print(y_train.shape)

In [43]:
def create_dataset(X_num, X_zip, y):
    # Numerical features: Float32
    x_num_t = torch.tensor(X_num, dtype=torch.float32)
    
    # Zip indices: Long (Int64). 
    # Shift by +1 so -1 (unknown) becomes 0.
    x_zip_t = torch.tensor(X_zip, dtype=torch.long) + 1
    
    # Targets: Long for loss calculation
    y_t = torch.tensor(y.values, dtype=torch.long)
    
    return TensorDataset(x_num_t, x_zip_t, y_t)


BATCH_SIZE = 1024
EPOCHS = 200
LEARNING_RATE = 1e-3

from sklearn.base import clone

def run_cv_grid_search(model_class, param_grid, X_raw, y_raw, n_splits=3):
    stratified_k_fold = StratifiedKFold(n_splits=n_splits, shuffle=True)
    parameter_grid = list(ParameterGrid(param_grid))
    results = [] 
    print(f"Starting Grid Search | {len(parameter_grid)} Combos | {n_splits}-Fold CV")

    for i, params in enumerate(parameter_grid):
        print(f"Fitting Combo {i+1}/{len(parameter_grid)}: {params}")
        
        # Store the metrics for the BEST epoch in each fold
        fold_baccs = [] 
        fold_accs = []
        fold_f1s = []
        fold_times = [] # Optional: Add timing here if you want

        for fold, (train_idx, val_idx) in enumerate(stratified_k_fold.split(X_raw, y_raw)):

            numerical_columns = [c for c in X_raw.columns if c not in embed_column]
            
            X_train_fold_raw = X_raw.iloc[train_idx]
            y_train_fold = y_raw.iloc[train_idx]
            
            X_val_fold_raw = X_raw.iloc[val_idx]
            y_val_fold = y_raw.iloc[val_idx]

            fold_num_pipe = clone(numerical_pipeline)
            fold_zip_pipe = clone(zip_pipeline)

            X_num_train = fold_num_pipe.fit_transform(X_train_fold_raw[numerical_columns], y_train_fold)
            X_zip_train = fold_zip_pipe.fit_transform(X_train_fold_raw[embed_column]).squeeze()

            X_num_val = fold_num_pipe.transform(X_val_fold_raw[numerical_columns])
            X_zip_val = fold_zip_pipe.transform(X_val_fold_raw[embed_column]).squeeze()

            
            fold_weights = compute_class_weight('balanced', classes=np.unique(y_train_fold), y=y_train_fold)
            fold_weights_tensor = torch.tensor(fold_weights, dtype=torch.float32).to(device)

            train_ds = create_dataset(X_num_train, X_zip_train, y_train_fold)
            val_ds = create_dataset(X_num_val, X_zip_val, y_val_fold)
            train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
            val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

            fold_params = params.copy()
            fold_params['cont_dim'] = X_num_train.shape[1]
    
            # --- Model Setup ---
            model = model_class(**fold_params).to(device)
            optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
            criterion = nn.CrossEntropyLoss(weight=fold_weights_tensor)
            
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='min', factor=0.1, patience=10, min_lr=1e-6
            )

            # --- TRACKING BEST SCORE FOR THIS FOLD ---
            best_fold_bacc = -float('inf') 
            best_fold_acc = 0.0
            best_fold_f1 = 0.0

            start_time = time.time()
            for epoch in range(EPOCHS):
                model.train()
                for b_cont, b_zip, b_y in train_loader:
                    b_cont, b_zip, b_y = b_cont.to(device), b_zip.to(device), b_y.to(device)
                    optimizer.zero_grad()
                    loss = criterion(model(b_cont, b_zip), b_y)
                    loss.backward()
                    optimizer.step()

                model.eval()
                val_loss = 0.0
                all_preds = []
                all_targets = []
                with torch.no_grad():
                    for b_cont, b_zip, b_y in val_loader:
                        b_cont, b_zip, b_y = b_cont.to(device), b_zip.to(device), b_y.to(device)
                        logits = model(b_cont, b_zip)
                        val_loss += criterion(logits, b_y).item()
                        
                        preds = torch.argmax(logits, dim=1)
                        all_preds.extend(preds.cpu().numpy())
                        all_targets.extend(b_y.cpu().numpy())

                avg_val_loss = val_loss / len(val_loader)
                
                # --- METRICS CALCULATION ---
                acc = accuracy_score(all_targets, all_preds)
                b_acc = balanced_accuracy_score(all_targets, all_preds)
                f1 = f1_score(all_targets, all_preds, average='weighted')
                
                scheduler.step(avg_val_loss)

                # Snapshot all metrics if Balanced Accuracy improves
                if b_acc > best_fold_bacc:
                    best_fold_bacc = b_acc
                    best_fold_acc = acc
                    best_fold_f1 = f1

            end_time = time.time()
            fold_times.append(end_time - start_time)

            # End of Fold: Record the SNAPSHOT metrics
            fold_baccs.append(best_fold_bacc)
            fold_accs.append(best_fold_acc)
            fold_f1s.append(best_fold_f1)
            
            print(f" Fold {fold+1} Best: B-Acc: {best_fold_bacc:.4f} | Acc: {best_fold_acc:.4f} | F1: {best_fold_f1:.4f}")

        # Average performance across the 3 folds
        mean_bacc = np.mean(fold_baccs)
        mean_acc = np.mean(fold_accs)
        mean_f1 = np.mean(fold_f1s)
        mean_time = np.mean(fold_times)
        
        full_params = {**params}
        
        # Store results in a dictionary structure similar to sklearn cv_results_
        results.append({
            'params': full_params, 
            'mean_test_balanced_acc': mean_bacc,
            'mean_test_acc': mean_acc,
            'mean_test_f1_weighted': mean_f1,
            'mean_fit_time': mean_time,
            'config_id': i 
        })
        
        print(f"Combo {i} Result: Mean B-Acc: {mean_bacc:.4f}\n")

    # --- Create DataFrame for Reporting ---
    df_results = pd.DataFrame(results)
    df_results = df_results.sort_values(by='mean_test_balanced_acc', ascending=False)
    
    print("\n" + "="*40)
    print("FINAL GRID SEARCH RESULTS")
    print("="*40)
    print(df_results[['config_id', 'mean_test_acc', 'mean_test_balanced_acc', 'mean_test_f1_weighted']])
    
    best_result = df_results.iloc[0]
    print(f"\nBest Hyperparams found (ID {best_result['config_id']}): {best_result['params']}")
    
    return best_result['params'], df_results

In [44]:
class FeedForwardModel(nn.Module):
    def __init__(self, cont_dim, hidden_dims=[128, 64, 32], output_dim=7):
        super().__init__()
        zip_embed_dim = 64
        num_zip_codes = 883 # borrower_address_zip 882 different values + 1 missing
        self.emb = nn.Embedding(num_zip_codes, zip_embed_dim)
        
        self.input_dim = cont_dim + zip_embed_dim

        layers = []
        in_dim = self.input_dim
        
        for h_dim in hidden_dims:
            layers.append(nn.Linear(in_dim, h_dim))
            layers.append(nn.BatchNorm1d(h_dim))
            layers.append(nn.LeakyReLU())
            layers.append(nn.Dropout(0.2))
            in_dim = h_dim
        
        self.mlp = nn.Sequential(*layers)
        
        # Projects the last hidden layer to the number of classes (7)
        self.head = nn.Linear(hidden_dims[-1], output_dim)

    def forward(self, X_cont, X_zip):
        # Embed zip codes
        # Result shape: (Batch_Size, zip_embed_dim)
        zip_embedded = self.emb(X_zip)
        
        # Concatenate continuous features + embeddings
        # Result shape: (Batch_Size, cont_dim + zip_embed_dim)
        x = torch.cat([X_cont, zip_embedded], dim=1)
        
        # Pass through MLP features
        x = self.mlp(x)
        
        # Final classification
        return self.head(x)

In [45]:
# the number of numerical features of the model is defined after training
# we need to pass it as a network parameter
input_cont_dim = X_train.shape[1]

param_grid = {
    # 'cont_dim': [input_cont_dim],
    'hidden_dims': [
        [128, 64, 32],      # Original
        [256, 128, 64, 32], # Deeper/Wider
        [64, 32]            # Shallower (prevent overfitting)
    ],
}

best_params, df_results = run_cv_grid_search(
    model_class=FeedForwardModel, 
    param_grid=param_grid, 
    X_raw=X_train, 
    y_raw=y_train, 
    n_splits=3
)

print("Finished GridSearchCV, best parameters:")
print(best_params)
print("---")
print(df_results.to_latex(index=False))

Starting Grid Search | 3 Combos | 3-Fold CV
Fitting Combo 1/3: {'hidden_dims': [128, 64, 32]}
 Fold 1 Best: B-Acc: 0.8586 | Acc: 0.8970 | F1: 0.8969


/Users/geko/unibo/data_analytics/project/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)


 Fold 2 Best: B-Acc: 0.8590 | Acc: 0.8991 | F1: 0.8992


/Users/geko/unibo/data_analytics/project/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)


 Fold 3 Best: B-Acc: 0.8562 | Acc: 0.8917 | F1: 0.8916
Combo 0 Result: Mean B-Acc: 0.8579

Fitting Combo 2/3: {'hidden_dims': [256, 128, 64, 32]}


...

 Fold 1 Best: B-Acc: 0.8559 | Acc: 0.8942 | F1: 0.8941
 Fold 2 Best: B-Acc: 0.8622 | Acc: 0.8943 | F1: 0.8941
 Fold 3 Best: B-Acc: 0.8617 | Acc: 0.8958 | F1: 0.8957
Combo 1 Result: Mean B-Acc: 0.8599

Fitting Combo 3/3: {'hidden_dims': [64, 32]}


/Users/geko/unibo/data_analytics/project/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)


 Fold 1 Best: B-Acc: 0.8452 | Acc: 0.8885 | F1: 0.8885


/Users/geko/unibo/data_analytics/project/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)


 Fold 2 Best: B-Acc: 0.8441 | Acc: 0.8896 | F1: 0.8898
 Fold 3 Best: B-Acc: 0.8397 | Acc: 0.8854 | F1: 0.8855
Combo 2 Result: Mean B-Acc: 0.8430


FINAL GRID SEARCH RESULTS
   config_id  mean_test_acc  mean_test_balanced_acc  mean_test_f1_weighted
1          1       0.894757                0.859939               0.894614
0          0       0.895937                0.857934               0.895923
2          2       0.887837                0.842988               0.887932

Best Hyperparams found (ID 1): {'hidden_dims': [256, 128, 64, 32]}
Finished GridSearchCV, best parameters:
{'hidden_dims': [256, 128, 64, 32]}
---
\begin{tabular}{lrrrrr}
\toprule
params & mean_test_balanced_acc & mean_test_acc & mean_test_f1_weighted & mean_fit_time & config_id \\
\midrule
{'hidden_dims': [256, 128, 64, 32]} & 0.859939 & 0.894757 & 0.894614 & 258.120205 & 1 \\
{'hidden_dims': [128, 64, 32]} & 0.857934 & 0.895937 & 0.895923 & 237.054176 & 0 \\
{'hidden_dims': [64, 32]} & 0.842988 & 0.887837 & 0.887932 & 